In [67]:
import pandas as pd 

In [68]:
df = pd.read_csv('../dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

On supprime les variables EmployeeCount, Over18 et StandardHours qui ont la même valeur pour tous les individus

In [69]:
df= df.drop(columns=["EmployeeCount", "Over18", "StandardHours"])

Changement du type d'Attrition en booléen (bool)
Changement du type d'OverTime en booléen (bool)
Changement du type de BusinessTravel en quantitative discrète (int64)

In [70]:
df.loc[df.Attrition == "No", "Attrition"] = 0
df.loc[df.Attrition == "Yes", "Attrition"] = 1
df.Attrition = df.Attrition.astype(bool)

df.loc[df.OverTime == "No", "OverTime"] = 0
df.loc[df.OverTime == "Yes", "OverTime"] = 1
df.OverTime = df.OverTime.astype(bool)

df.loc[df.BusinessTravel == "Non-Travel", "BusinessTravel"] = 1
df.loc[df.BusinessTravel == "Travel_Rarely", "BusinessTravel"] = 2
df.loc[df.BusinessTravel == "Travel_Frequently", "BusinessTravel"] = 3
df.BusinessTravel = df.BusinessTravel.astype(np.int64)

Changement des variables de type object en type category (concerne seulement les variables qualitatives)

In [71]:
df.Department = pd.Categorical(df.Department, categories=["Research & Development", "Sales", "Human Resources"])
df.EducationField = pd.Categorical(df.EducationField, categories=["Life Sciences", "Medical", "Marketing", "Technical Degree", "Other", "Human Resources"])
df.Gender = pd.Categorical(df.Gender, categories=["Male", "Female"])
df.JobRole = pd.Categorical(df.JobRole, categories=["Sales Executive", "Research Scientist", "Laboratory Technician", "Manufacturing Director", "Healthcare Representative", "Manager", "Sales Representative", "Research Director", "Human Resources"])
df.MaritalStatus = pd.Categorical(df.MaritalStatus, categories=["Married", "Single", "Divorced"])

Export du nouveau jeu de données

In [72]:
df.to_csv("../dataset/data_cleaned.csv", index=False)

Création d'un deuxième jeu de données tranformant les varibales quantitatives discrètes avec peu de valeurs possibles (ex : 1 'Low'; 2 'Medium'; 3 'High'; 4 'Very High') en variable qualitative (category) afin de permettre des visualisations différentes des variables

In [73]:
df.BusinessTravel = pd.Categorical(df.BusinessTravel.replace({1: "Non-Travel", 2: "Travel_Rarely", 3: "Travel_Frequently"}),categories=["Non-Travel", "Travel_Rarely", "Travel_Frequently"], ordered = True)

# Ici ce sont les variables de la description du jeu de données sur kaggme
categories_education = ["Below College", "College", "Bachelor", "Master", "Doctor"]
categories_environment_satisfaction = ["Low", "Medium", "High", "Very High"]
categories_job_involvement = ["Low", "Medium", "High", "Very High"]
categories_job_level = ["Entry Level", "Junior Level", "Mid Level", "Senior Level", "Executive Level"]
categories_job_satisfaction = ["Low", "Medium", "High", "Very High"]
categories_performance_rating = ["Low", "Good", "Excellent", "Outstanding"]
categories_relationship_satisfaction = ["Low", "Medium", "High", "Very High"]
categories_work_life_balance = ["Bad", "Good", "Better", "Best"]
df["Education"] = pd.Categorical(df["Education"].replace({1: "Below College", 2: "College", 3: "Bachelor", 4: "Master", 5: "Doctor"}), ordered=True, categories=categories_education)
df["EnvironmentSatisfaction"] = pd.Categorical(df["EnvironmentSatisfaction"].replace({1: "Low", 2: "Medium", 3: "High", 4: "Very High"}), ordered=True, categories=categories_environment_satisfaction)
df["JobInvolvement"] = pd.Categorical(df["JobInvolvement"].replace({1: "Low", 2: "Medium", 3: "High", 4: "Very High"}), ordered=True, categories=categories_job_involvement)
df["JobLevel"] = pd.Categorical(df["JobLevel"].replace({1: "Entry Level", 2: "Junior Level", 3: "Mid Level", 4: "Senior Level", 5: "Executive Level"}), ordered=True, categories=categories_job_level)
df["JobSatisfaction"] = pd.Categorical(df["JobSatisfaction"].replace({1: "Low", 2: "Medium", 3: "High", 4: "Very High"}), ordered=True, categories=categories_job_satisfaction)
df["PerformanceRating"] = pd.Categorical(df["PerformanceRating"].replace({1: "Low", 2: "Good", 3: "Excellent", 4: "Outstanding"}), ordered=True, categories=categories_performance_rating)
df["RelationshipSatisfaction"] = pd.Categorical(df["RelationshipSatisfaction"].replace({1: "Low", 2: "Medium", 3: "High", 4: "Very High"}), ordered=True, categories=categories_relationship_satisfaction)
df["WorkLifeBalance"] = pd.Categorical(df["WorkLifeBalance"].replace({1: "Bad", 2: "Good", 3: "Better", 4: "Best"}), ordered=True, categories=categories_work_life_balance)

# Rajout de variables qui présentent peu de valeurs possibles et qui pourraient être interessant d'étudier comme des variables qualitatives
df.TrainingTimesLastYear = pd.Categorical(df.TrainingTimesLastYear,categories=[0,1,2,3,4,5,6], ordered = True)
df.StockOptionLevel = pd.Categorical(df.StockOptionLevel,categories=[0,1,2,3], ordered = True)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,True,Travel_Rarely,1102,Sales,1,College,Life Sciences,1,Medium,...,Excellent,Low,0,8,0,Bad,6,4,0,5
1,49,False,Travel_Frequently,279,Research & Development,8,Below College,Life Sciences,2,High,...,Outstanding,Very High,1,10,3,Better,10,7,1,7
2,37,True,Travel_Rarely,1373,Research & Development,2,College,Other,4,Very High,...,Excellent,Medium,0,7,3,Better,0,0,0,0
3,33,False,Travel_Frequently,1392,Research & Development,3,Master,Life Sciences,5,Very High,...,Excellent,High,0,8,3,Better,8,7,3,0
4,27,False,Travel_Rarely,591,Research & Development,2,Below College,Medical,7,Low,...,Excellent,Very High,1,6,3,Better,2,2,2,2


Export du jeu de données

In [74]:
df.to_csv("../dataset/data_cleaned_quali.csv", index=False)